In [28]:
#%pip install tf-keras

import numpy as np
import pandas as pd
import joblib

from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.corpus import stopwords

In [29]:
# 1) Cargar datos
books_df = pd.read_csv("../data/raw/goodreads_data.csv")

# Combinar stopwords de nltk con otras posibles palabras irrelevantes
my_stop_words = {'to-read', 'closed', 'abandoned-books'
'rating-top', 'literature', 'not-interested', 'libricos', 'recommended',
'000-next', 
'001-ladder-top', '002-ladder-short-term', '003-ladder-medium-term', '004-ladder-long-term', 
'005-ladder-maybe-someday', 
'_cristina', '_giorgia', '_natalia', '_nieves', '_pilar', '_sindy', '_víctor', 'chełmińska'
}
stop_words = set(my_stop_words)
stop_words = stop_words.union(set(stopwords.words('english')))
stop_words = stop_words.union(set(stopwords.words('spanish')))
stop_words = stop_words.union(set(stopwords.words('french')))
stop_words = stop_words.union(set(stopwords.words('italian')))

# 3. Procesar los tags: convertirlos en listas y filtrar stop-words
def filter_stopwords(tags):
    """
    Filtra las stop-words de una lista de etiquetas.
    """
    return [tag for tag in tags if tag.lower() not in stop_words]


books_df.rename(columns={
    "Book": "book_title",
    "Description": "blurb",
    "Genres": "tags"
}, inplace=True)

books_df["tags"] = books_df["tags"].fillna("[]").apply(
    lambda x: ", ".join(
        tag.strip().lower().replace(" ", "-") for tag in eval(x)
    )
)

books_df = books_df.sample(1000, random_state=42)

books_df["book_title"] = books_df["book_title"].fillna("")
books_df["blurb"] = books_df["blurb"].fillna("")
books_df["tags"] = books_df["tags"].fillna("")
books_df["text"] = books_df["book_title"] + ". " + books_df["blurb"]


# Filtyer stopwords from text
books_df['tags'] = books_df['tags'].apply(lambda x: filter_stopwords([t.strip() for t in str(x).split(',')]))
books_df['text'] = books_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

In [30]:
# 7) Cargar Sentence-BERT y vectorizar
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') 
X = model.encode(books_df['text'].tolist(), show_progress_bar=True)

# 8) Binarizar las etiquetas
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(books_df['tags'])

# 9) Separar train y test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 10) Construir el Random Forest en modo multi-output
rf_base = RandomForestClassifier(random_state=42)
multi_rf = MultiOutputClassifier(rf_base)

# 11) Definir la grilla de hiperparámetros
param_grid = {
    'estimator__n_estimators': [5, 10],  # Incrementar el número de árboles
    'estimator__max_depth': [1, 2],     # Probar con mayor profundidad
    'estimator__min_samples_leaf': [1, 2],   # Ajustar el tamaño mínimo de hojas
    'estimator__class_weight': ['balanced']     # Balancear clases automáticamente
}

# 2) Cambiar la métrica de evaluación
grid_search = GridSearchCV(
    multi_rf,
    param_grid=param_grid,
    cv=2,  # Incrementar el número de folds
    scoring='f1_macro',  # Cambiar a f1_macro para multietiqueta
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Fitting 2 folds for each of 8 candidates, totalling 16 fits


GridSearchCV(cv=2,
             estimator=MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)),
             n_jobs=-1,
             param_grid={'estimator__class_weight': ['balanced'],
                         'estimator__max_depth': [1, 2],
                         'estimator__min_samples_leaf': [1, 2],
                         'estimator__n_estimators': [5, 10]},
             scoring='f1_macro', verbose=2)

In [31]:
print("Número de etiquetas originales:", len(mlb.classes_))
print("Etiquetas originales:", mlb.classes_)

Número de etiquetas originales: 354
Etiquetas originales: ['' '17th-century' '18th-century' '19th-century' '20th-century'
 '21st-century' 'abuse' 'action' 'adult' 'adult-fiction' 'adventure'
 'africa' 'african-american' 'algeria' 'aliens' 'amazon' 'american'
 'american-civil-war' 'american-history' 'ancient' 'ancient-history'
 'angels' 'animal-fiction' 'animals' 'anthologies' 'anthropology'
 'apocalyptic' 'architecture' 'art' 'art-history' 'arthurian'
 'artificial-intelligence' 'asia' 'asian-literature' 'astrology' 'atheism'
 'audiobook' 'australia' 'autistic-spectrum-disorder' 'autobiography'
 'banned-books' 'bdsm' 'beauty-and-the-beast' 'biblical-fiction'
 'biography' 'biography-memoir' 'biology' 'book-club' 'books-about-books'
 'british-literature' 'buddhism' 'buisness' 'business' 'canada' 'catholic'
 'cats' 'chapter-books' 'chick-lit' 'childrens' 'china' 'christian'
 'christian-fantasy' 'christian-fiction' 'christian-living'
 'christian-non-fiction' 'christian-romance' 'christianit

In [32]:
# Evaluar el mejor modelo
best_rf = grid_search.best_estimator_

# 13) Evaluar en test
test_score = best_rf.score(X_test, y_test)
print("Puntuación final en test:", test_score)
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor puntuación (CV):", grid_search.best_score_)

Puntuación final en test: 0.0
Mejores hiperparámetros: {'estimator__class_weight': 'balanced', 'estimator__max_depth': 1, 'estimator__min_samples_leaf': 1, 'estimator__n_estimators': 10}
Mejor puntuación (CV): 0.04225092355999792


In [33]:
# Haz la prueba con un libro
test_book = "The Great Gatsby. A novel set in the 1920s, exploring themes of wealth, love, and the American Dream."
expected_tags = ["classics", "fiction", "literature", "novel", "romance", "historical-fiction", "american", "20th-century", "great-american-novels"]

# Vectorizar el texto del libro
test_vector = model.encode([test_book])

# Predecir etiquetas con el modelo
predicted_binary = best_rf.predict(test_vector)  # Esto devuelve una matriz binaria
predicted_tags = mlb.inverse_transform(predicted_binary)[0]  # Convertir a etiquetas originales

# Mostrar resultados coloreando en ver los aciertos
print("\nEtiquetas esperadas:")
for tag in expected_tags:
    if tag in predicted_tags:
        print(f"\033[92m{tag}\033[0m", end=", ")  # Verde para aciertos
    else:
        print(f"{tag}", end=", ")  # Rojo para fallos
print("\n\nEtiquetas predichas:")
for tag in predicted_tags:
    if tag in expected_tags:
        print(f"\033[92m{tag}\033[0m", end=", ")  # Verde para aciertos
    else:
        print(f"\033[91m{tag}\033[0m", end=", ")  # Rojo para fallos
print("\n\n")


Etiquetas esperadas:
classics, fiction, literature, novel, romance, historical-fiction, american, 20th-century, great-american-novels, 

Etiquetas predichas:
adventure, american, animals, audiobook, childrens, classics, contemporary, fantasy, fiction, historical, historical-fiction, humor, nonfiction, novels, school, womens, 




In [34]:
# 14) Guardar modelo y binarizador si es necesario
joblib.dump(best_rf, "../model/book_tagging_rf.joblib")
joblib.dump(mlb, "../model/book_tagging_rf_mlb.joblib")

['../model/book_tagging_rf_mlb.joblib']